In [3]:
import pandas as pd

# load dataset
df = pd.read_csv("../data/data.csv")

# basic checks
df.head()


,Job Title,Description
0,Data Analyst,job overview were seeking a data analyst to tu...
1,Data Reporting Analyst,about wspc wspc is a cooperative of outstandin...
2,Data Analyst (Power BI/Python),data analyst power bipython employment type fu...
3,Data & Reporting Analyst,our company pharmerica overview the data repor...
4,Data Quality Analyst (Remote Opportunity),vetsez is seeking a data quality analyst teste...


In [4]:
print("Columns:", df.columns.tolist())
print("Number of rows:", len(df))


Columns: ['Job Title', 'Description']
Number of rows: 521


In [5]:
df['Description'].iloc[0]

'job overview were seeking a data analyst to turn reports from dispatch, safety, accounting, and maintenance into actionable insights and visual dashboards. youll help drive smarter decisions across our trucking operations by translating data into strategies that improve efficiency and performance. key responsibilities analyze data across departments to uncover trends and recommend improvements build and maintain dashboards to track kpis cost per mile, fuel efficiency, driver performance, etc. collaborate with team leads to identify reporting needs and deliver insights conduct etl processes to clean and prepare data present findings clearly via reports and visualizations requirements proficient in excel and power bi, tableau or similar tools strong data analysis and critical thinking skills experience with sql and etl processes preferred bonus familiarity with python or r excellent communication skillsable to simplify complex data for stakeholders understanding of logistics or trucking

In [6]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PRISHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()                      # 1. lowercase
    text = re.sub(r'[^a-z\s]', '', text)     # 2. remove punctuation & numbers
    words = text.split()                     # 3. tokenize
    words = [w for w in words if w not in stop_words]  # 4. remove stopwords
    return " ".join(words)


In [8]:
df.head()

,Job Title,Description
0,Data Analyst,job overview were seeking a data analyst to tu...
1,Data Reporting Analyst,about wspc wspc is a cooperative of outstandin...
2,Data Analyst (Power BI/Python),data analyst power bipython employment type fu...
3,Data & Reporting Analyst,our company pharmerica overview the data repor...
4,Data Quality Analyst (Remote Opportunity),vetsez is seeking a data quality analyst teste...
